In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import warnings
warnings.simplefilter('ignore')

In [3]:
image_directory = None
depth_directory = None
video_path = None
json_path = None
csv_path = None
txt_path = None

weights_dir_path = '../weights'
optical_flow_checkpoint = os.path.join('/Vol0/user/f.konokhov/tfoptflow/tfoptflow/pwcnet-lg-6-2-cyclic-KITTI_finetuned',
                                       'pwcnet.ckpt-84000')

depth_estimator_name = 'struct2depth'
depth_checkpoint = os.path.join(weights_dir_path, 'model-199160')
#depth_estimator_name = 'senet'

computation_kwargs = dict(
    cuda_visible_devices=0,
)

sys.path.append(os.path.join(os.path.abspath(os.pardir), 'submodules/tf_models/research/struct2depth'))
sys.path.append(os.path.join(os.path.abspath(os.pardir), 'submodules/tfoptflow/tfoptflow'))

In [4]:
from prepare_dataset.dataset_builder import ImagesDatasetBuilder

### Initialize

In [5]:
sequence_directory = 'meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned'

builder = ImagesDatasetBuilder(
     sequence_directory=sequence_directory,
     build_from='DIRECTORY',
     image_directory='/dbstore/datasets/retail_bot/meetingroom_04_rgbd_ir_imu_pose/data_rgb',
#    txt_path=txt_path,
     mode=ImagesDatasetBuilder.TEST,
     estimate_optical_flow=True,
     optical_flow_estimator_name='pwc',
     optical_flow_checkpoint=optical_flow_checkpoint,
     estimate_depth=False,
     depth_estimator_name=depth_estimator_name,
     depth_checkpoint=depth_checkpoint,
     memory_safe=True,
     **computation_kwargs)
builder._configure()

Using FPS rate = 30


### Build

In [6]:
builder.build() # "Do everything" big red button

Using FPS rate = 30
Estimate optical flow
Model:   <class 'prepare_dataset.estimator.PWCOpticalFlowEstimator'>
Weights: /Vol0/user/f.konokhov/tfoptflow/tfoptflow/pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-84000
Output:  meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned/optical_flow_stride1


Computation settings:



Using TensorFlow backend.


CUDA visible devices:     0
Available GPUs:           /job:localhost/replica:0/task:0/device:GPU:0
Allow growth:             True
GPU memory fraction:      0.33
Number of CPU:            8
Number of CPU threads:    16

Random seed:              42



Building model...
... model built.
Loading model checkpoint /Vol0/user/f.konokhov/tfoptflow/tfoptflow/pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-84000 for eval or testing...

INFO:tensorflow:Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-84000


I0603 15:49:30.195708 47366295576192 tf_logging.py:115] Restoring parameters from /Vol0/user/f.konokhov/tfoptflow/tfoptflow/pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-84000


... model loaded
Estimator(dir=meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned/optical_flow_stride1, image_manager=ImageManager(dir=/dbstore/datasets/retail_bot/meetingroom_04_rgbd_ir_imu_pose/data_rgb, image_height=480, image_width=640, stride=1, sample=True, step=1), checkpoint=/Vol0/user/f.konokhov/tfoptflow/tfoptflow/pwcnet-lg-6-2-cyclic-KITTI_finetuned/pwcnet.ckpt-84000)


Optical flow estimation: 100%|██████████| 5262/5262 [16:09<00:00,  4.92it/s]


In [7]:
builder.dataframe.to_csv(sequence_directory + '/df.csv')

### Test Visual Odometry

In [8]:
target_size = (120, 160)

In [10]:
from prepare_dataset.generator_factory import GeneratorFactory
dataset = GeneratorFactory(
    csv_name=('df.csv'),
    dataset_root='/Vol0/user/i.slynko/projects/odometry/notebooks',
    train_sequences=[sequence_directory],
    val_sequences=[sequence_directory],
    target_size=target_size,
    x_col=['path_to_optical_flow'],
    y_col=['euler_x', 'euler_y', 'euler_z', 'x', 'y', 'z'],
    image_columns=['path_to_optical_flow'],
    load_modes=['flow_xy'],
    preprocess_modes=['flow_xy'],
    val_sampling_step=1,
    cached_imgs=None
)

100%|██████████| 1/1 [00:00<00:00, 35.62it/s]


No cache


In [11]:
dataset.df_val.path_to_optical_flow = dataset.df_val.path_to_optical_flow.str.replace(
    '/{}/'.format(sequence_directory), '/')

In [18]:
from models.model_factory import ModelFactory

model_factory = ModelFactory(
    target_size,
    channels_counts=dataset.channels_counts,
    lr=0.001,
    loss='mae',
    scale_rotation=50
)
model = model_factory.construct_pretrained_model(
    '/Vol0/user/f.konokhov/FlavorNet/models/suncg_v04/weights/models-suncg_v04-ls_vo_no_decoder_kitti_flow.110-0.000146.hdf5')

Load weights from /Vol0/user/f.konokhov/FlavorNet/models/suncg_v04/weights/models-suncg_v04-ls_vo_no_decoder_kitti_flow.110-0.000146.hdf5


In [19]:
generator = dataset.get_val_generator()
model_output = model.predict_generator(generator, steps=len(generator))

No cache


In [20]:
# TODO: recheck naming
# list of dicts is overcomplecated!

predictions = {'yaw':   model_output[0].flatten(), 
               'pitch': model_output[1].flatten(), 
               'roll':  model_output[2].flatten(), 
               'tx':    model_output[3].flatten(), 
               'ty':    model_output[4].flatten(),
               'tz':    model_output[5].flatten()}

predictions_transposed = [dict(zip(predictions, col)) for col in zip(*predictions.values())]

In [21]:
sys.path.append('/Vol0/user/i.slynko/projects/FlavorNet/utilities/convertors')

In [22]:
from trajectory import RelativeTrajectory
trajectory = RelativeTrajectory()
trajectory.from_euler_angles(predictions_transposed) # from_euler_angles should return self (see DataFrame)
trajectory.relative_to_global().plot('tmp.html')

In [23]:
# Old visualization:
sys.path.append('/Vol0/user/i.slynko/projects')
sys.path.append('/Vol0/user/i.slynko/projects/FlavorNet/utilities/slam_metrics')
from FlavorNet.models.predict import test
test('tmp', model, dataset)

tmp
No cache
trajectory meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned
saved 6-dof to predictions/tmp/6dof/meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned.csv
saved global se3 to predictions/tmp/se3/meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned.txt
saved global poses to predictions/tmp/poses/meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned.txt
